## **Динамическое ценообразование - A / B-тестирование - Моделирование продаж**


##### В этом проекте я попытался применить A / B-тестирование, чтобы определить оптимальную цену с доверительным интервалом и смоделировать продажи с разными ценами. Наша цель - обеспечить максимальную прибыль, как и предполагалось :)

#### Данные, которые мы собираемся использовать, состоят из двух столбцов: Shop_id и Price.
#### Shop_id представляет 6 различных магазинов, а Price - это сумма, которую покупатель желает заплатить за конкретный товар.
#### Набор данных создан на основе ответов на опрос, проведенный в разных магазинах.
#### В этом проекте считается, что участники опроса рационально ответили на вопросы.

### **Загразку библиотек**

In [ ]:
import pandas as pd
from scipy import stats
from scipy.stats import shapiro
import statsmodels.stats.api as sms


### **Подготовка данных**

In [ ]:
df= pd.read_csv("data.csv")

In [ ]:
df.head()

,shop_id,price
0,shop_4,32.117753
1,shop_3,30.711370
2,shop_3,31.572607
3,shop_4,34.543840
4,shop_4,47.205824


In [ ]:
df.shop_id.value_counts()

shop_4    1661
shop_6     733
shop_3     615
shop_2     144
shop_5     129
shop_1      97
Name: shop_id, dtype: int64

##### Как мы видим, здесь участники опроса не сбалансированы между магазинами. Но мы продолжим, как будто это уравновешено

##### Давайте посмотрим, какова средняя цена в разных магазина

In [ ]:
df.groupby("shop_id").agg({"price": "mean"})

,price
shop_id,
shop_1,36.175498
shop_2,35.693170
shop_3,35.477261
shop_4,43.872913
shop_5,37.443592
shop_6,40.376575


In [ ]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
price,3379.0,40.771387,19.730179,10.0,30.0,30.0,30.0,31.802522,34.744389,40.569827,56.921335,74.453248,135.848659,205.052944


##### Когда мы смотрим на дисперсию ценовой переменной, мы видим, что наше предположение относительно ответов участников опроса кажется правильным, за некоторыми исключениями.


##### Избавимся от цен выше 100

In [ ]:
df=df[df.price <= 100]

In [ ]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
price,3292.0,38.284972,11.657494,10.0,30.0,30.0,30.0,31.672294,34.678475,38.528301,53.188295,65.198083,88.622812,99.169972


##### Разделяем магазины для тестирования

In [ ]:
shop_1 = df[(df["shop_id"] == "shop_1")]
shop_2 = df[(df["shop_id"] == "shop_2")]
shop_3 = df[(df["shop_id"] == "shop_3")]
shop_4 = df[(df["shop_id"] == "shop_4")]
shop_5 = df[(df["shop_id"] == "shop_5")]
shop_6 = df[(df["shop_id"] == "shop_6")]


### **A/B Testing**

##### Чтобы выбрать параметрический или непараметрический тест, нам необходимо проверить распределение и вариативность значений каждого магазина.
##### Если значения нормально распределены. Мы будем использовать независимый t-тест (параметрический). В противном случае мы будем использовать U-критерий Манна – Уитни (непараметрический).


#### **Тест нормального распределения**

##### Наши гипотезы
##### H0 = Ценовая переменная имеет нормальное распределение
##### H1 = Ценовая переменная обычно не распределяется

In [ ]:
shop_list = [shop_1,shop_2,shop_3,shop_4,shop_5,shop_6]
for i in shop_list:
    test_statistics, pvalue = shapiro(i["price"])
    print('Test statistics = %.4f, p-value = %.4f' % (test_statistics, pvalue))

Test statistics = 0.6190, p-value = 0.0000
Test statistics = 0.5513, p-value = 0.0000
Test statistics = 0.4935, p-value = 0.0000
Test statistics = 0.7639, p-value = 0.0000
Test statistics = 0.6382, p-value = 0.0000
Test statistics = 0.6631, p-value = 0.0000


##### Мы отвергли гипотезу H0. Потому что, как вы можете видеть, p-значения меньше "0,05".

##### В этом случае необходимо использовать U-критерий Манна – Уитни. Если данные не распределены нормально, нам не нужно проверять однородность дисперсии.

#### Mann–Whitney U test

##### Наши гипотезы
##### H0 = Статистически нет разницы между ценами в магазинах
##### H1 = По статистике существует разница между ценами в магазинах

In [ ]:
shop_list = [shop_1,shop_2,shop_3,shop_4,shop_5,shop_6] 
shop_names = ["shop_1","shop_2","shop_3","shop_4","shop_5","shop_6"] 
test_stat = []
p_values = []
first_shop =[]
second_shop = []
for i in range(0, len(shop_list)):
    for j in range(0, len(shop_list)):
        if i == j:                     
            continue
        test_statistics, pvalue = stats.mannwhitneyu(shop_list[i]["price"], shop_list[j]["price"])
        test_stat.append(test_statistics)
        p_values.append(pvalue.round(4))
        first_shop.append(shop_names[i]) 
        second_shop.append(shop_names[j])
        
p_value_table = pd.DataFrame({"first_shop":first_shop,"second_shop":second_shop,"test_stat": test_stat, "p_values":p_values})


##### Давайте посмотрим, как выглядит наша таблица p_value

In [ ]:
p_value_table

,first_shop,second_shop,test_stat,p_values
0,shop_1,shop_2,5010.0,0.0001
1,shop_1,shop_3,29424.0,0.4252
2,shop_1,shop_4,60158.0,0.0001
3,shop_1,shop_5,6121.0,0.3905
4,shop_1,shop_6,34006.0,0.4124
5,shop_2,shop_1,5010.0,0.0001
6,shop_2,shop_3,31313.0,0.0000
7,shop_2,shop_4,65110.0,0.0000
8,shop_2,shop_5,6575.5,0.0000
9,shop_2,shop_6,36586.0,0.0000


##### Как видите, в таблице представлены все возможные комбинации сравнения цен между магазинами.
##### Если вы думаете о дублированных записях выше, не волнуйтесь, это было специально.
##### Мы будем использовать таблицу как есть

In [ ]:
p_value_table.groupby("first_shop").agg({"p_values": "mean"})

,p_values
first_shop,
shop_1,0.24566
shop_2,0.00002
shop_3,0.20790
shop_4,0.00004
shop_5,0.24426
shop_6,0.22760


##### Здесь, как вы можете видеть, мы взяли среднее значение p, чтобы понять, насколько эти магазины отличаются друг от друга по цене.
##### Если значение p меньше «0,05», мы можем сказать, что существует значительная разница между магазинами, как мы заявили в нашем гипотезе.
##### Основываясь на этой информации, по статистике shop_2 и shop_4 отличаются от других магазинов по цене. И эта разница не может быть случайной.
##### По другим магазинам нельзя сказать.
##### Теперь мы знаем, что два вышеупомянутых магазина отличаются от других магазинов ценой, которую покупатели готовы платить.
#### Поскольку наша цель - обеспечить максимальную прибыль, нам необходимо иметь разные цены в этих двух магазинах, поскольку они не похожи ни на один из магазинов, включая друг друга (проверьте строку 7 таблицы значений p).
#### Для других магазинов, которые статистически не отличаются друг от друга по цене, мы можем установить такую ​​же цену.
### В соответствии с политикой компании мы можем решить, что цена товара должна быть одинаковой во всех магазинах. Но мы этого делать не будем.




### **Доверительные интервалы**

##### Поскольку мы решили продолжить с разными ценами, нам нужно определить разные доверительные интервалы.

In [ ]:
alike_shops = ["shop_1", "shop_3","shop_5", "shop_6"] # одинаковые магазинов с цегами
shop_1356 = df[(df["shop_id"].isin(alike_shops))] # здесь мы собираем их, чтобы вычислить доверительный интервал
shop_1356_con_int =sms.DescrStatsW(shop_1356["price"]).tconfint_mean() # здесь вычисляется доверительный интервал
shop_1356_con_int

(35.803488556367476, 36.7997387818449)

##### Здесь мы рассчитали доверительный интервал для одинаковых магазинов.

In [ ]:
shop_2_con_int = sms.DescrStatsW(shop_2["price"]).tconfint_mean() 
shop_2_con_int

(33.244221031281384, 36.24683640338621)

##### Здесь мы рассчитали доверительный интервал для shop_2

In [ ]:
shop_4_con_int = sms.DescrStatsW(shop_4["price"]).tconfint_mean()
shop_4_con_int

(39.891734916786355, 41.15449478737718)

##### Здесь мы рассчитали доверительный интервал для shop_4

### **Моделирование прибыли**

#### Рассчитаем прибыль с минимальным и максимальным значениями доверительных интервалов.
#### Сначала рассчитаем прибыль с максимальным значением доверительных интервалов.

##### Shop_1356 Расчет выручки

In [ ]:
freq_1356 = len(shop_1356[shop_1356["price"] >= shop_1356_con_int[1]]) # здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал
revenue_1356 = freq_1356 * shop_1356_con_int[1] # здесь мы умножили то, что нашли выше, на максимальный доверительный интервал, чтобы рассчитать доход
print(f'Revenue: {revenue_1356}')

Revenue: 10524.725291607641


##### Расчет выручки Shop_2

In [ ]:
freq_2 = len(shop_2[shop_2["price"] >= shop_2_con_int[1]]) # здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал
revenue_2 = freq_2 * shop_2_con_int[1] # здесь мы умножили то, что нашли выше, на максимальный доверительный интервал, чтобы рассчитать доход
print(f'Revenue: {revenue_2}')

Revenue: 724.9367280677243


##### Shop_1356 Расчет выручки

In [ ]:
freq_4 = len(shop_4[shop_4["price"] >= shop_4_con_int[1]]) # здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал
revenue_4 = freq_4 * shop_4_con_int[1] # здесь мы умножили то, что нашли выше, на максимальный доверительный интервал, чтобы рассчитать доход
print(f'Revenue: {revenue_4}')

Revenue: 20412.629414539082


In [ ]:
total_revenue_with_max = revenue_1356 + revenue_2 + revenue_4 # здесь вычисляем общий доход
total_revenue_with_max.round(2)

31662.29

##### Как видите, наш общий доход составляет 31662,29 для сценария, в котором мы решили использовать максимальное значение доверительного интервала

##### **Теперь посчитаем прибыль с минимальным значением доверительных интервалов.**

##### Shop_1356 Расчет выручки

In [ ]:
freq_1356 = len(shop_1356[shop_1356["price"] >= shop_1356_con_int[0]]) #  здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал 
revenue_1356 = freq_1356 * shop_1356_con_int[0] # здесь мы умножили то, что нашли на максимальный доверительный интервалб чтобы рассчитать доход
print(f'Revenue: {revenue_1356}')

Revenue: 12602.827971841352


##### Расчет выручки Shop_2

In [ ]:
freq_2 = len(shop_2[shop_2["price"] >= shop_2_con_int[0]]) # здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал 
revenue_2 = freq_2 * shop_2_con_int[0] # здесь мы умножили то, что нашли на максимальный доверительный интервалб чтобы рассчитать доход
print(f'Revenue: {revenue_2}')

Revenue: 1662.2110515640693


##### Shop_1356 Расчет выручки

In [ ]:
freq_4 = len(shop_4[shop_4["price"] >= shop_4_con_int[0]]) # здесь мы определили, сколько людей готовы платить больше, чем наш максимальный доверительный интервал
revenue_4 = freq_4 * shop_4_con_int[0] # здесь мы умножили то, что нашли выше, на максимальный доверительный интервал, чтобы рассчитать доход
print(f'Revenue: {revenue_4}')

Revenue: 21780.88726456535


In [ ]:
total_revenue_with_min = revenue_1356 + revenue_2 + revenue_4 # вычисляем общий доход
total_revenue_with_min.round(2)

36045.93

##### Как видите, наш общий доход составляет 36045,93 для сценария, в котором мы решили использовать минимальное значение доверительного интервала.

### **Выводы**

##### В этом проекте нашей целью было установить оптимальную цену на конкретный товар, который продается в разных магазинах.
##### Мы оценили результаты опроса, применив A / B-тест, чтобы увидеть, есть ли статистически значимая разница между ценами, которые люди хотят платить в магазинах.
##### Согласно результатам A / B-теста, мы обнаружили, что 2 магазина не похожи ни на какие другие.
##### Затем мы определили доверительные интервалы для разных магазинов.
##### И наконец, моделируем продажи по максимальному и минимальному значениям доверительного интервала.
##### Согласно нашим выводам, приведенным выше, установка цены на минимальное значение доверительного интервала привело к увеличению суммы дохода.
##### Однако это не означает, что этот продукт выгоднее продавать с минимальным значением доверительного интервала, поскольку мы не знаем, сколько стоит этот продукт.
##### Но наша работа здесь сделана. Мы провели необходимые тесты и предоставили необходимую информацию.